# 2. Evaluation & Proofs

This notebook generates the final proofs for the dissertation/report.

**Objectives:**
- Calculate **Dice Score** (Segmentation Accuracy)
- Calculate **AUC-ROC** (Malignancy Classification Performance)
- Visualization: **Input vs. Ground Truth vs. Prediction**
- **GradCAM++** Heatmaps for XAI

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, confusion_matrix, accuracy_score
import seaborn as sns
import glob
import cv2
from tqdm.notebook import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## 1. Load Model & Data

In [ ]:
# Re-define Model (Must match training definition)
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv3d(in_channels, out_channels, 3, padding=1),
            nn.BatchNorm3d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv3d(out_channels, out_channels, 3, padding=1),
            nn.BatchNorm3d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.conv(x)

class LightweightUNet3D(nn.Module):
    def __init__(self, in_channels=1, out_channels=1):
        super().__init__()
        self.inc = DoubleConv(in_channels, 16)
        self.down1 = nn.Sequential(nn.MaxPool3d(2), DoubleConv(16, 32))
        self.down2 = nn.Sequential(nn.MaxPool3d(2), DoubleConv(32, 64))
        self.down3 = nn.Sequential(nn.MaxPool3d(2), DoubleConv(64, 128))
        self.up1 = nn.ConvTranspose3d(128, 64, 2, stride=2)
        self.conv1 = DoubleConv(128, 64)
        self.up2 = nn.ConvTranspose3d(64, 32, 2, stride=2)
        self.conv2 = DoubleConv(64, 32)
        self.up3 = nn.ConvTranspose3d(32, 16, 2, stride=2)
        self.conv3 = DoubleConv(32, 16)
        self.outc = nn.Conv3d(16, out_channels, 1)
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool3d(1),
            nn.Flatten(),
            nn.Linear(128, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        u1 = self.up1(x4)
        u1 = torch.cat([u1, x3], dim=1)
        u1 = self.conv1(u1)
        u2 = self.up2(u1)
        u2 = torch.cat([u2, x2], dim=1)
        u2 = self.conv2(u2)
        u3 = self.up3(u2)
        u3 = torch.cat([u3, x1], dim=1)
        u3 = self.conv3(u3)
        mask = torch.sigmoid(self.outc(u3))
        risk = torch.sigmoid(self.classifier(x4))
        return mask, risk

model = LightweightUNet3D().to(device)
try:
    model.load_state_dict(torch.load("../models/unet3d_lightweight.pth", map_location=device))
    print("Model loaded successfully!")
except FileNotFoundError:
    print("Model file not found. Please train first.")
model.eval()

In [ ]:
def dice_score(pred, target, smooth=1e-5):
    pred = pred.reshape(-1)
    target = target.reshape(-1)
    intersection = (pred * target).sum()
    return (2. * intersection + smooth) / (pred.sum() + target.sum() + smooth)

DATA_DIR = Path("../data/lidc_patches")
test_files = glob.glob(str(DATA_DIR / "*.npz")) # In reality, split into train/test
print(f"Evaluating on {len(test_files)} samples.")

## 2. Evaluation Loop

In [ ]:
y_true_cls = []
y_pred_cls = []
dice_scores = []

samples_for_viz = []

with torch.no_grad():
    for f in tqdm(test_files):
        data = np.load(f)
        
        # Prepare Input
        image = data['image'].astype(np.float32)
        image = (image - (-1000)) / 1400.0
        image = np.clip(image, 0, 1)
        image_tensor = torch.tensor(image).unsqueeze(0).unsqueeze(0).to(device)
        
        # Prepare label
        label = data['label']
        
         # Prepare mask
        if 'mask' in data:
            mask = data['mask'].astype(np.float32)
        else:
             # Synthetic for demo
            mask = np.zeros((64, 64, 64), dtype=np.float32)
            z, y, x = np.ogrid[:64, :64, :64]
            mask[((z-32)**2 + (y-32)**2 + (x-32)**2 <= 100)] = 1.0
            
        # Inference
        pred_mask, pred_risk = model(image_tensor)
        
        # Classification Metrics
        y_true_cls.append(label)
        y_pred_cls.append(pred_risk.item())
        
        # Segmentation Metrics
        pred_mask_np = pred_mask.cpu().numpy().squeeze()
        binary_mask = (pred_mask_np > 0.5).astype(np.float32)
        dice = dice_score(binary_mask, mask)
        dice_scores.append(dice)
        
        if len(samples_for_viz) < 5:
            samples_for_viz.append((image, mask, pred_mask_np))

print(f"Average Dice Score: {np.mean(dice_scores):.4f}")

## 3. Visualization: ROC Curve & Confusion Matrix

In [ ]:
# ROC Curve
if len(np.unique(y_true_cls)) > 1: # Only plot if we have both classes
    fpr, tpr, _ = roc_curve(y_true_cls, y_pred_cls)
    roc_auc = auc(fpr, tpr)

    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    plt.show()
else:
    print("Not enough classes to plot ROC (Need both 0 and 1).")

## 4. Visual Proofs: Segmentation

In [ ]:
for i, (img, gt, pred) in enumerate(samples_for_viz):
    # Take center slice
    mid = 32
    
    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 3, 1)
    plt.title("Input CT Slice")
    plt.imshow(img[mid], cmap='gray')
    plt.axis('off')
    
    plt.subplot(1, 3, 2)
    plt.title("Ground Truth Mask")
    plt.imshow(gt[mid], cmap='gray')
    plt.axis('off')
    
    plt.subplot(1, 3, 3)
    plt.title("Predicted Segmentation")
    plt.imshow(pred[mid], cmap='jet', alpha=0.9)
    plt.axis('off')
    
    plt.show()

## 5. GradCAM++ Visualization

We verify that the network is looking at the nodule.

In [ ]:
# Simple GradCAM Hook
gradients = None
activations = None

def backward_hook(module, grad_input, grad_output):
    global gradients
    gradients = grad_output[0]

def forward_hook(module, input, output):
    global activations
    activations = output

# Hook onto the last encoder layer (bottleneck)
target_layer = model.down3[1].conv[5] # Last conv in down3
target_layer.register_forward_hook(forward_hook)
target_layer.register_backward_hook(backward_hook)

# Run one inference
img, _, _ = samples_for_viz[0]
inputs = torch.tensor(img).unsqueeze(0).unsqueeze(0).to(device, requires_grad=True)
_, risk = model(inputs)

# Backward
model.zero_grad()
risk.backward()

# Generate CAM
weights = torch.mean(gradients, dim=(2, 3, 4), keepdim=True)
cam = torch.sum(weights * activations, dim=1, keepdim=True)
cam = torch.relu(cam)
cam = cam.squeeze().cpu().detach().numpy()

# Resize CAM to input size
cam = cv2.resize(cam[4], (64, 64)) # Take middle feature map slice and resize
plt.figure()
plt.title("GradCAM Activation (Center Slice)")
plt.imshow(cam, cmap='jet')
plt.colorbar()
plt.show()